In [1]:
### IMPORTS ###
# Quantum libraries:
import pennylane as qml
from pennylane import numpy as np
import autograd.numpy as anp

# Plotting
from matplotlib import pyplot as plt

# Other
import copy
from tqdm.notebook import tqdm # Pretty progress bars
from IPython.display import Markdown, display # Better prints
import joblib # Writing and loading
##############


In [2]:
from qfuncs import *

In [3]:
dev = qml.device('default.qubit', wires=8, shots=1000)

def convolution_8(params,N=8):
    k = 0
    
    for wire, wire_next in zip(range(0,N,2), range(1,N,2)):
        qml.RY(params[k], wires = wire)
        k = k + 1
        
        qml.CNOT(wires=[wire, wire_next])
        
    qml.Barrier()
    
    for wire, wire_next in zip(range(N-2,0,-2), range(N-3,-1,-2)):
        qml.RY(params[k], wires = wire_next)
        k = k + 1
        
        qml.CNOT(wires=[wire_next, wire])

        
def pooling_8(params, N=8):
    qml.Barrier()
    
    k = N
    # First pooling 8->4
    for wire_meas, wire_next in zip(range(0,N,2), range(1,N,2)):
        m_0 = qml.measure(wire_meas)

        qml.cond(m_0 ==0, qml.RY)(params[k], wires=wire_next)
        k = k + 1
        qml.cond(m_0 ==1, qml.RY)(params[k], wires=wire_next)
        k = k + 1
    
    qml.Barrier()
    
    # Second pooling 4->2
    for wire_meas, wire_next in zip(range(1,N,4), range(3,N,4)):
        m_0 = qml.measure(wire_meas)

        qml.cond(m_0 ==0, qml.RY)(params[k], wires=wire_next)
        k = k + 1
        qml.cond(m_0 ==1, qml.RY)(params[k], wires=wire_next)
        k = k + 1
        
    qml.Barrier()
    
    qml.RY(params[k], wires = 3)
    k = k + 1
    qml.RY(params[k], wires = 7)
    k = k + 1
    
    # Third pooling 2->1
    for wire_meas, wire_next in zip(range(3,N,8), range(7,N,4)):
        m_0 = qml.measure(wire_meas)

        qml.cond(m_0 ==0, qml.RY)(params[k], wires=wire_next)
        k = k + 1
        qml.cond(m_0 ==1, qml.RY)(params[k], wires=wire_next)
        k = k + 1
        
@qml.qnode(dev)        
def qcnn_8(params_vqe,params):
    vqe_ising_chain_circuit(params_vqe, 8)
    qml.Barrier()
    convolution_8(params)
    pooling_8(params)
    
    return qml.probs(wires=7)

In [4]:
def cross_entropy_loss(labels, predictions):
    '''
    f = - Σ [(Yi*log(Pi) + (1 - Yi)*log(1 - Pi) )]  for i in TrainingSet
    '''
    loss = 0
    for l, p in zip(labels, predictions):
        term = ( l * np.log(p) + (1 - l) * np.log(1 - p) )
        loss = loss + term

    return -1 * loss

In [44]:
def cost_fn(params, X, Y):
    predictions = [qcnn_8(x, params) for x in X]
    loss = cross_entropy(Y, predictions)
    
    return loss

In [45]:
def qcnn_training(X_train, Y_train, steps = 100, lr = 0.001, batch_size = 25, N = 8, plot = True):
    # Initialize parameters randomly
    params = np.random.randn(3*N)
    
    # Set optimizer
    opt = qml.GradientDescentOptimizer(stepsize=lr)
    
    loss_history = []
    batch_size = min(len(X_train), batch_size)
    
    progress = tqdm(range(steps))
    for epoch in progress:
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]

        # Compute current loss and update parameters
        params, cost_new = opt.step_and_cost(lambda v: cost_fn(v, X_batch, Y_batch), params)
        
        loss_history.append(cost_new)
        
        # Set description to current loss
        progress.set_description('Cost: {0}'.format(np.round(cost_new,3)))

    if plot:
        plt.figure(figsize=(15,5))
        plt.plot(np.arange(steps), loss_history)
        plt.title('Loss history')
        plt.ylabel('Cross entropy')
        plt.xlabel('Epoch')
        plt.grid(True)
        
    return loss_history, params

In [46]:
drawer = qml.draw(qcnn_8)
print(drawer([0]*100,[0]*100))

0: ──RY(0.00)─╭C────────────────────||──RY(0.00)──||──RY(0.00)─╭C──||───────────────||
1: ──RY(0.00)─╰X─╭C─────────────────||──RY(0.00)──||───────────╰X──||──RY(0.00)─╭C──||
2: ──RY(0.00)────╰X─╭C──────────────||──RY(0.00)──||──RY(0.00)─╭C──||───────────╰X──||
3: ──RY(0.00)───────╰X─╭C───────────||──RY(0.00)──||───────────╰X──||──RY(0.00)─╭C──||
4: ──RY(0.00)──────────╰X─╭C────────||──RY(0.00)──||──RY(0.00)─╭C──||───────────╰X──||
5: ──RY(0.00)─────────────╰X─╭C─────||──RY(0.00)──||───────────╰X──||──RY(0.00)─╭C──||
6: ──RY(0.00)────────────────╰X─╭C──||──RY(0.00)──||──RY(0.00)─╭C──||───────────╰X──||
7: ──RY(0.00)───────────────────╰X──||──RY(0.00)──||───────────╰X──||───────────────||

──╭C─────────────────────────╭C──────────────────────────||───────────────────────────
──╰ControlledOperation(0.00)─╰ControlledOperation(0.00)──||─╭C────────────────────────
──╭C─────────────────────────╭C──────────────────────────||─│─────────────────────────
──╰ControlledOperation(0.00)─╰ControlledOp

In [47]:
data = joblib.load('./params.job')

In [48]:
X_train = np.array(data[0][1])
Y_train = np.array(data[0][2])

for J in range(1,len(data)):
    X_train = np.concatenate((X_train, np.array(data[J][1])))
    Y_train = np.concatenate((Y_train, np.array(data[J][2])))
    
print('Size of Training Set: {0}'.format(np.shape(X_train)[0]))

Size of Training Set: 50


In [49]:
def cross_entropy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        c_entropy = l * (anp.log(p[l])) + (1 - l) * anp.log(1 - p[1 - l])
        loss = loss + c_entropy

    return -1 * loss

In [50]:
cross_entropy_loss(Y_train, Y_train + 1)

tensor(nan, requires_grad=True)

In [ ]:
loss_history, params = qcnn_training(X_train, Y_train, lr = 1e-2, steps = 300)

  0%|          | 0/300 [00:00<?, ?it/s]

In [41]:
drawer = qml.draw(qcnn_8)
print(drawer(X_train[2],params))

0: ──RY(2.10)─╭C────────────────────||──RY(0.87)──||──RY(-0.87)─╭C──||────────────────||─╭C────────
1: ──RY(1.17)─╰X─╭C─────────────────||──RY(1.12)──||────────────╰X──||──RY(0.04)──╭C──||─╰RY(-0.99)
2: ──RY(1.23)────╰X─╭C──────────────||──RY(1.19)──||──RY(0.01)──╭C──||────────────╰X──||─╭C────────
3: ──RY(1.25)───────╰X─╭C───────────||──RY(1.18)──||────────────╰X──||──RY(-0.90)─╭C──||─╰RY(0.19)─
4: ──RY(1.26)──────────╰X─╭C────────||──RY(1.17)──||──RY(-0.99)─╭C──||────────────╰X──||─╭C────────
5: ──RY(1.25)─────────────╰X─╭C─────||──RY(1.21)──||────────────╰X──||──RY(-0.79)─╭C──||─╰RY(0.30)─
6: ──RY(1.24)────────────────╰X─╭C──||──RY(1.27)──||──RY(-0.16)─╭C──||────────────╰X──||─╭C────────
7: ──RY(1.23)───────────────────╰X──||──RY(1.37)──||────────────╰X──||────────────────||─╰RY(-0.32)

──╭C──────────||───────────────────────||────────────────────────────────┤       
──╰RY(-0.56)──||─╭C─────────╭C─────────||────────────────────────────────┤       
──╭C──────────||─│──────────│──────